In [1]:
#==task5 7 GBDT+LR 推荐算法 ==

# 前面介绍的协同过滤和矩阵分解存在的劣势就是仅利用了用户与物品相互行为信息进行推荐， 忽视了用户自身特征， 物品自身特征以及上下文信息等，导致生成的结果往往会比较片面。
# 2014年由Facebook提出的GBDT+LR模型
# 利用【GBDT自动进行特征筛选和组合， 进而生成新的离散特征向量】【GBDT:梯度提升决策树】， 再把【该特征向量当做LR模型的输入】， 来产生最后的【预测】结果，
# 该模型能够综合利用用户、物品和上下文等多种不同的特征， 生成较为全面的推荐结果， 在CTR点击率预估场景下使用较为广泛。



In [3]:
#以下分别介绍
# 7.2逻辑回归模型
# 逻辑回归模型非常重要， 在推荐领域里面， 相比于传统的协同过滤， 逻辑回归模型能够综合利用用户、物品、上下文等。
# 在线性回归的基础上加了一个 Sigmoid 函数（非线形）映射，变分类算法。
# 逻辑回归假设数据服从伯努利分布,通过极大化似然函数的方法，运用梯度下降来 求解参数，来达到将数据二分类的目的。
# 相比于协同过滤和矩阵分解利用用户的物品“相似度”进行推荐， 逻辑回归模型将问题看成了一个分类问题， 通过预测正样本的概率对物品进行排序。
# 正样本可以是用户“点击”了某个商品或者“观看”了某个视频， 均是推荐系统希望用户产生的“正反馈”行为。
# 因此逻辑回归模型将推荐问题转化成了一个点击率预估问题。就是一个典型的二分类。

# LR如何做？
# 1. 将用户年龄、性别、物品属性、物品描述、当前时间、当前地点等特征转成数值型向量。
# 2. 确定逻辑回归的优化目标，比如把点击率预测转换成二分类问题， 这样就可以得到分类问题常用的损失作为目标，训练模型。
# 3. 在预测的时候， 将特征向量输入模型产生预测， 得到用户“点击”物品的概率。
# 4. 利用点击概率对候选物品排序， 得到推荐列表。
# 关键就是每个特征的权重参数W,我们一般是使用梯度下降的方式，首先会先随机初始化参数W ， 然后将特征向量（也就是我们上面数值化出来的特征）输入到模型,
# 通过计算得到模型的预测概率， 然后通过对目标函数求导得到每个W,的梯度， 然后进行更新W,通过若干次迭代， 就可以得到最终的W了,敲定模型公式。下次拿去预测就是差不多准确可用的。

# 优缺点
# 1. LR模型形式简单，可解释性好，从特征的权重可以看到不同的特征对最后结果的影响。
# 2. 训练时便于并行化，在预测时只需要对特征进行线性加权，所以性能比较好，往往适合处理海量id类特征，可以防止信息损失（相对于范化的CTR特征）。
# 3. 资源占用小,尤其是内存。实际的工程应用中只需要存储权重比较大的特征及特征对应的权重。
# 4. 方便输出结果调整。很易的对这些概率分数进行cutoff，也就是划分阈值(大于某个阈值的是一类，小于某个阈值的是一类)

# 缺点
# 1. 表达能力不强， 无法进行特征交叉， 无法进行特征交叉，特征筛选等一系列“高级”操作需要人工来做，需经验，可能造成损失。
# 2. 准确率并不是很高。个线性模型加了个sigmoid， 形式非常的简单(非常类似线性模型)，很难去拟合数据的真实分布。
# 3. 处理非线性数据较麻烦。如果想处理非线性， 首先对连续特征的处理需要先进行离散化（离散化的目的是为了引入非线性）。人工分桶的方式会引入多
# 种问题。
# 4. LR 需要进行人工特征组合，需经验，的模型迁移起来比较困难，换一个领域又需要重新进行大量的特征工程。

# 所以如何自动发现有效的特征、特征组合，弥补人工经验不足，缩短LR特征实验周期？
# GBDT模型， 正好可以自动发现特征并进行有效组合

In [4]:
# 7.3GBDT模型
# 梯度提升决策树
# 在传统机器学习算法里面是对真实分布拟合的最好的几种算法之一，在前几年深度学习还没有大行其道之前，gbdt在各种竞赛是大放异彩。
# 一是效果确实挺不错。二是即可以用于【分类也可以用于回归。三是可以筛选特征】。所以这个模型依然是一个非常重要的模型。
# GBDT是通过采用加法模型(即基函数的线性组合），以及不断减小训练过程产生的误差来达到将数据分类或者回归的算法：
# gbdt通过多轮迭代， 每轮迭代会产生一个弱分类器， 每个分类器在上一轮分类器的残差（每轮迭代的时候，弱分类器的输出的结果相减）基础上进行训练。     
# gbdt对弱分类器的要求一般是足够简单， 并且低方差（数据差异小）高偏差（与均值差）。因为训练的过程是通过降低偏差来不断提高最终分类器的精度。
# 所以每个分类回归树的深度不会很深。
# 最终的总分类器是将每轮训练得到的弱分类器加权求和得到的（也就是加法模型）。
# gbdt无论用于分类还是回归一直都是使用的CART回归树。
# 通过不断构建决策树的方式，使预测结果一步步的接近目标值。
# 是二分类问题和回归问题的损失函数不同，
# 回归问题中一般使用的是平方损失，二分类问题中， GBDT和逻辑回归一样，//sigmoid相关,

# 构建分类GBDT的步骤:
# 1. 初始化GBDT
# 2. 循环生成决策树:
# 这一步有4小步， 第一就是计算负梯度值得到残差， 第二步是用回归树拟合（靠近）残差，第三步是计算叶子节点的输出值， 第四步是更新模型。
# 通过多次循环迭代， 就可以得到一个比较强的学习器。

# GBDT的优缺点：
# 可以把树的生成过程理解成自动进行多维度的特征组合的过程，从根结点到叶子节点上的整个路径(多个特征值判断)，才能最终决定一棵树的预测值，
# 【连续型特征的处理，GBDT 可以拆分出一个临界阈值，比如大于 0.027 走左子树，小于等于 0.027（或者 default 值）走右子树，
# 这样很好的规避了人工离散化的问题，
# 通过这里解决了逻辑回归不能自动发现特征并进行有效组合的问题， 这也是GBDT的优势所在。】

# 局限性：
# 海量的id类特征，由于树的深度和棵数限制（防止过拟合），不能有效的存储；
# 也会存在性能瓶颈， one hot 特征大于 10 万维时，就必须做分布式的训练才能保证不爆内存。
# GBDT 通常配合少量的反馈 CTR 特征来表达，虽然具有一定的范化能力，但是同时会有信息损失，对于头部资源不能有效的表达（LR刚好相反）。

# 所以， 我们发现其实GBDT和LR的优缺点可以进行互补。


In [5]:
# 7.4 GBDT+LR模型

# 2014年， Facebook提出了一种利用GBDT自动进行特征筛选和组合， 进而生成新的离散特征向量， 
# 再把该特征向量当做LR模型的输入， 来产生最后的预测结果， 这就是著名的GBDT+LR模型了。
# GBDT+LR 使用最广泛的场景是CTR点击率预估，即预测当给用户推送的广告会不会被用户点击。

# 训练时，GBDT 建树的过程相当于自动进行的特征组合和离散化，
# 然后从根结点到叶子节点的这条路径就可以看成是不同特征进行的特征组合，用叶子节点可以唯一的表示这条路径，
# 并作为一个离散特征传入 LR 进行二次训练。

# 如：有两棵树，x为一条输入样本，遍历两棵树，左树有三个叶子节点，右树有两个叶子节点，最终的特征即为五维的向量。对于输入x，假设他落在左树第二个节点，编码[0,1,0]，落在右
# 树第二个节点则编码[0,1]，所以整体的编码为[0,1,0,0,1]，这类编码作为特征，输入到线性分类模型（LR or FM）中进行分类。
# 预测时，会先走 GBDT 的每棵树，得到某个叶子节点对应的一个离散特征(即一组特征组合)，
# 然后把该特征以 one-hot 形式传入 LR 进行线性加权预测。

# 几点注意：
# 1. 通过GBDT进行特征组合之后得到的离散向量是和训练数据的原特征一块作为逻辑回归的输入， 而不仅仅全是这种离散特征。
# 2. 建树的时候用ensemble建树的原因就是一棵树的表达能力很弱，不足以表达多个有区分性的特征组合，
# 多棵树的表达能力更强一些。GBDT每棵树都在学习前面棵树尚存的不足，迭代多少次就会生成多少棵树。
# 3. RF也是多棵树，但从效果上有实践证明不如GBDT。且GBDT前面的树，特征分裂主要体现对多数样本有区分度的特
# 征；后面的树，主要体现的是经过前N颗树，残差仍然较大的少数样本。优先选用在整体上有区分度的特征，再选用
# 针对少数样本有区分度的特征，思路更加合理，这应该也是用GBDT的原因。
# 4. 在CRT预估中， GBDT一般会建立两类树(非ID特征建一类， ID类特征建一类)， AD，ID类特征在CTR预估中是非常
# 重要的特征，直接将AD，ID作为feature进行建树不可行，故考虑为每个AD，ID建GBDT树。
# 4.1. 非ID类树：不以细粒度的ID建树，此类树作为base，即便曝光少的广告、广告主，仍可以通过此类树得到有区分性的特征、特征组合。
# 4.2. ID类树：以细粒度 的ID建一类树，用于发现曝光充分的ID对应有区分性的特征、特征组合。

In [6]:
# ==7.5 编程实践==
# 模型架构： 首先是要训练GBDT模型， GBDT的实现一般可以使用xgboost， 或者lightgbm。训练完了
# GBDT模型之后， 我们需要预测出每个样本落在了哪棵树上的哪个节点上， 然后通过one-hot就会得到一些新的离散特征，
# 这和原来的特征进行合并组成新的数据集， 然后作为逻辑回归的输入，最后通过逻辑回归模型得到结果。


In [9]:
#处理好数据x_train, y_train
# ==1. 训练GBDT模型==
# GBDT模型的搭建我们可以通过XGBOOST， lightgbm等进行构建。
gbm = lgb.LGBMRegressor(objective='binary',
 subsample= 0.8,
 min_child_weight= 0.5,
 colsample_bytree= 0.7,
 num_leaves=100, 
 max_depth = 12,
 learning_rate=0.05,
 n_estimators=10,
 )

gbm.fit(x_train, y_train,
 eval_set = [(x_train, y_train), (x_val, y_val)],
 eval_names = ['train', 'val'],
 eval_metric = 'binary_logloss',
# early_stopping_rounds = 100,
 )

In [ ]:
# ==2. 特征转换并构建新的数据集==
# 通过上面我们建立好了一个gbdt模型， 我们接下来要用它来预测出样本会落在每棵树的哪个叶子节点上， 为后面的离
# 散特征构建做准备， 注意不是用gbdt预测结果而是预测训练数据在每棵树上的具体位置：
model = gbm.booster_ # 获取到建立的树
# 每个样本落在每个树的位置 ， 下面两个是矩阵 (样本个数, 树的棵树) ， 每一个数字代表某个样本落在了某个数的哪个叶子节点
gbdt_feats_train = model.predict(train, pred_leaf = True)
gbdt_feats_test = model.predict(test, pred_leaf = True)
# 把上面的矩阵转成新的样本-特征的形式， 与原有的数据集合并
gbdt_feats_name = ['gbdt_leaf_' + str(i) for i in range(gbdt_feats_train.shape[1])]
df_train_gbdt_feats = pd.DataFrame(gbdt_feats_train, columns = gbdt_feats_name) 
df_test_gbdt_feats = pd.DataFrame(gbdt_feats_test, columns = gbdt_feats_name)
# 构造新数据集
train = pd.concat([train, df_train_gbdt_feats], axis = 1)
test = pd.concat([test, df_test_gbdt_feats], axis = 1)
train_len = train.shape[0]
data = pd.concat([train, test])

In [ ]:
# ==3. 离散特征的独热编码，并划分数据集==
# 新数据的新特征进行独热编码
for col in gbdt_feats_name:
    onehot_feats = pd.get_dummies(data[col], prefix = col)
    data.drop([col], axis = 1, inplace = True)
    data = pd.concat([data, onehot_feats], axis = 1)
# 划分数据集
train = data[: train_len]
test = data[train_len:]
x_train, x_val, y_train, y_val = train_test_split(train, target, test_size = 0.3, random_state 
= 2018)

In [ ]:
# ==4. 训练逻辑回归模型作最后的预测==
# 训练逻辑回归模型
lr = LogisticRegression()
lr.fit(x_train, y_train)
tr_logloss = log_loss(y_train, lr.predict_proba(x_train)[:, 1])
print('tr-logloss: ', tr_logloss)
val_logloss = log_loss(y_val, lr.predict_proba(x_val)[:, 1])
print('val-logloss: ', val_logloss)
# 预测
y_pred = lr.predict_proba(test)[:, 1]

In [ ]:
# 7.6课后思考
# 1. 为什么使用集成的决策树？ 为什么使用GBDT构建决策树而不是随机森林？ 
# //只为得到简单自动拆分分组特征，生成离散向量，不是最终分类，要简单，浅
# 2. 面对高维稀疏类特征的时候(比如ID类特征)， 逻辑回归一般要比GBDT这种非线性模型好， 为什么？
# //因为树复杂